In [7]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import cv2
import tkinter
from tkinter import filedialog
from PIL import Image, ImageTk

class App:
    window = tkinter.Tk()
    
    def __init__(self):
        self.showWindow()
    
    def showWindow(self):
        take_photo=tkinter.Button(self.window, text="Take a photo", width=50, command=self.displayPhoto)
        take_photo.pack(anchor=tkinter.CENTER, expand=True)
        take_photo=tkinter.Button(self.window, text="Upload image", width=50, command=self.uploadImage)
        take_photo.pack(anchor=tkinter.CENTER, expand=True)
        self.window.mainloop()
        
    def findEdges(self, image):
        edged = cv2.Canny(image, 10, 250)
        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (7, 7))
        closed = cv2.morphologyEx(edged, cv2.MORPH_CLOSE, kernel)
        (cnts, _) = cv2.findContours(closed.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        for c in cnts:
            peri = cv2.arcLength(c, True)
            approx = cv2.approxPolyDP(c, 0.02 * peri, True)
            cv2.drawContours(image, [approx], -1, (0, 255, 0), 2)
        return image
    
    def findEdges2(self, img):
        hsv_min = np.array((2, 28, 65), np.uint8)
        hsv_max = np.array((26, 238, 255), np.uint8)
        hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV )
        thresh = cv2.inRange( hsv, hsv_min, hsv_max )
        contours, hierarchy = cv2.findContours( thresh.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        cv2.drawContours( img, contours, -1, (255,0,0), 3, cv2.LINE_AA, hierarchy, 1 )
        return img
    
    def findEdges3(self, img):
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        blur = cv2.GaussianBlur(gray,(1,1),1000)
        flag, thresh = cv2.threshold(blur, 120, 255, cv2.THRESH_BINARY)
        # Find contours
        contours, hierarchy = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
        contours = sorted(contours, key=cv2.contourArea,reverse=True) 
        # Select long perimeters only
        perimeters = [cv2.arcLength(contours[i],True) for i in range(len(contours))]
        listindex=[i for i in range(15) if perimeters[i]>perimeters[0]/2]
        numcards=len(listindex)
        # Show image
        imgcont = img.copy()
        [cv2.drawContours(imgcont, [contours[i]], 0, (0,255,0), 5) for i in listindex]
        cv2.imshow("Image", imgcont)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    
    def capturePhoto(self):
        cam = cv2.VideoCapture(0)
        cv2.namedWindow("Image")

        while True:
            ret, frame = cam.read()
            cv2.imshow("Image", frame)
            if not ret:
                break
            k = cv2.waitKey(1)
            cam.release()
            cv2.destroyAllWindows()
            return frame
         
    def displayPhoto(self):
        img = self.capturePhoto()
        edges = self.findEdges3(img)
        cv2.imshow("Image", edges)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        
    def uploadImage(self):
        img_path = filedialog.askopenfilename(filetypes=[('Image Files', ['.jpeg', '.jpg', '.png', '.gif',
                                                           '.tiff', '.tif', '.bmp'])])
        img = cv2.imread(img_path)
        edges = self.findEdges3(img)
        cv2.imshow("Image", edges)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

In [8]:
app = App()

Exception in Tkinter callback
Traceback (most recent call last):
  File "/home/kirill/anaconda3/lib/python3.7/tkinter/__init__.py", line 1705, in __call__
    return self.func(*args)
  File "<ipython-input-7-a221e347efd0>", line 86, in uploadImage
    cv2.imshow("Image", edges)
cv2.error: OpenCV(4.1.2) /io/opencv/modules/highgui/src/window.cpp:376: error: (-215:Assertion failed) size.width>0 && size.height>0 in function 'imshow'

